In [3]:
import numpy as np
import pandas as pd
import os
# for dirname,_,filenames in os.walk('kaggle/input'):
#   for filename in filenames:
#     print(os.path.join(diename, filename))
from keras.layers.core.dropout import Dropout
from keras.layers.core.dense import Dense
from keras.layers.core.flatten import Flatten
from keras.layers.core.activation import Activation
from keras.layers.pooling import MaxPooling2D
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv2D,MaxPool2D,AveragePooling2D,Input,BatchNormalization, MaxPooling2D, Activation, Flatten, Dense, Dropout
from keras.models import Sequential
from keras.utils import np_utils
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler
from keras_preprocessing import image
import scipy
import os
import cv2
import keras_metrics as km

callbacks = [
    keras.callbacks.TensorBoard(
        log_dir='./mylog/',
        histogram_freq=1,
    )
]


    
data = pd.read_csv("./anger.csv")

#x数据集是图片的像素；y数据集是标签
x_data = data["pixels"]
y_data = data["emotion"]
sns.set_theme(style="darkgrid")
oversampler = RandomOverSampler(sampling_strategy="auto")

#数据集预处理
x_data, y_data = oversampler.fit_resample(x_data.values.reshape(-1,1),y_data)
# x_data = x_data.values.reshape(-1, 1)
# y_data = y_data.values.reshape(-1, 1)
print(x_data.shape, " ", y_data.shape)
y_data.value_counts()
y_data.shape
x_data = pd.Series(x_data.flatten())
x_data = np.array(list(map(str.split, x_data)), np.float32)
x_data /= 255
# x_data[:10]
x_data = x_data.reshape(-1, 48, 48, 1)
print(x_data.shape)
y_data = np.array(y_data)
y_data = y_data.reshape(y_data.shape[0], 1)
print(y_data.shape)
x_train,x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.25, random_state=45)

model = Sequential([ 
  Input((48, 48, 1)),
  Conv2D(32, kernel_size=(3,3), strides=(1,1), padding='valid'),
  BatchNormalization(axis=3),
  Activation('relu'),
  Conv2D(64, (3,3), strides=(1,1), padding='same'),
  BatchNormalization(axis=3),
  Activation('relu'),
  MaxPooling2D((2,2)),
  Conv2D(64, (3, 3), strides=(1,1), padding='valid'),
  BatchNormalization(axis=3),
  Activation('relu'),
  Conv2D(128,(3, 3), strides=(1, 1), padding='same'),
  BatchNormalization(axis=3),
  Activation('relu'),
  MaxPooling2D((2, 2)),
  Conv2D(128, (3, 3), strides=(1, 1), padding='valid'),
  BatchNormalization(axis=3),
  Activation('relu'),
  MaxPooling2D((2,2)),
  Flatten(),
  Dense(200, activation='relu'),
  Dropout(0.6),
  Dense(1,activation='sigmoid')  # 如果是一分类表情识别， 此处为1；
  ])
model.summary()
adam = keras.optimizers.adam_v2.Adam(learning_rate=0.0001)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy',km.f1_score(),km.binary_precision(),km.binary_recall()])
# y_train = np_utils.to_categorical(y_train, 2)
# y_train.shape
# y_test = np_utils.to_categorical(y_test, 2)
# y_test.shape
#模型训练
history = model.fit(x_train,y_train,epochs=15,validation_data=(x_test, y_test),callbacks=[callbacks])

(61868, 1)   (61868,)
(61868, 48, 48, 1)
(61868, 1)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 32)        320       
                                                                 
 batch_normalization (BatchN  (None, 46, 46, 32)       128       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 46, 46, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 46, 46, 64)        18496     
                                                                 
 batch_normalization_1 (Batc  (None, 46, 46, 64)       256       
 hNormalization)                                                 
                                                                 
 act

2022-04-15 20:37:18.888698: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/ywq/anaconda3/envs/FER/lib/python3.8/site-packages/cv2/../../lib64:
2022-04-15 20:37:18.888750: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/15


/home/ywq/anaconda3/envs/FER/lib/python3.8/site-packages/keras_metrics/metrics.py:26: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return object.__getattribute__(self, name)


1451/1451 [==============================] - 186s 126ms/step - loss: 0.6405 - accuracy: 0.6344 - f1_score: 0.5946 - precision: 0.5978 - recall: 0.5917 - val_loss: 0.5414 - val_accuracy: 0.7292 - val_f1_score: 0.6470 - val_precision: 0.6480 - val_recall: 0.6461
Epoch 2/15
1451/1451 [==============================] - 179s 123ms/step - loss: 0.5180 - accuracy: 0.7398 - f1_score: 0.6759 - precision: 0.6764 - recall: 0.6754 - val_loss: 0.4547 - val_accuracy: 0.7835 - val_f1_score: 0.6990 - val_precision: 0.6994 - val_recall: 0.6986
Epoch 3/15
1451/1451 [==============================] - 177s 122ms/step - loss: 0.4182 - accuracy: 0.8058 - f1_score: 0.7182 - precision: 0.7182 - recall: 0.7181 - val_loss: 0.3693 - val_accuracy: 0.8384 - val_f1_score: 0.7373 - val_precision: 0.7354 - val_recall: 0.7391
Epoch 4/15
1451/1451 [==============================] - 177s 122ms/step - loss: 0.3120 - accuracy: 0.8666 - f1_score: 0.7550 - precision: 0.7515 - recall: 0.7584 - val_loss: 0.2931 - val_accuracy

In [28]:
import keras
import keras_metrics as km
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model_accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','test'],loc='upper left')
plt.show()
#summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model_loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'],loc='upper left')
plt.show()
#summarize history for f1
plt.plot(history.history['km.f1_score']
plt.title('f1_score')
plt.ylabel('f1_score')
plt.xlabel('epoch')
# plt.legend(['train','test'],loc='upper left')
plt.show()
# y_pred=model.predict(x_test)
# y_result=[]
# for pred in y_pred:
#   y_result.append(np.argmax(pred))
# print(y_result[:])
# y_actual=[]
# for pred in y_test:
#   y_actual.append(np.argmax(pred))
# print(y_actual[:])

# from sklearn.metrics import confusion_matrix, classification_report
# print(classification_report(y_actual,y_result))

# import seaborn as sn
# cm = tf.math.confusion_matrix(labels=y_actual,predictions=y_result)


# plt.figure(figsize=(10,7))
# sn.heatmap(cm,annot=True,fmt='d')
# plt.xlabel('Predicted')
# plt.ylabel('Truth')
# fer_json=model.to_json()
# with open("fer_json", "w") as json_file:
#   json_file.write(fer_json)

# #保存模型
# model.save_weights("./anger_model.h5")


SyntaxError: invalid syntax (1475640501.py, line 20)